In [4]:
import numpy as np
import imageio
import cv2
import os

def PCA(x, k):
    old_m = x.shape[0]
    old_n = x.shape[1]
    meanvalue = np.mean(x, axis=0)
    x = x-meanvalue

    cov_x = np.cov(x)

    eigen_values, eigen_vectors = np.linalg.eig(cov_x)

    index = np.argsort(-eigen_values)
    eigen_values = eigen_values[index]
    eigen_vectors = eigen_vectors[:, index]

    sum = eigen_values.sum()
    p = sum * k
    temp = 0
    cnt = 0
    while temp < p:
        temp = temp + eigen_values[cnt]
        cnt = cnt + 1
    x1 = eigen_vectors[:, range(cnt)]
    x2 = np.dot(x1.T, x)
    new_m = x2.shape[0]
    new_n = x2.shape[1]

    new_x = np.dot(x1, x2) + meanvalue
    new_x = new_x.astype(np.uint8)

    new_x = new_x * (new_x >= 0)
    new_x = new_x * (new_x < 256)
    
    err=np.linalg.norm(new_x-x)
    print(err)
    
    com = (1.0*new_m*new_n)/(old_m*old_n)
    print(com)

    return new_x

def row(path):
    image = imageio.imread(path)
    m = image.shape[0]
    n = image.shape[1]

    if(m != 256 or n != 256):
        gap_m = 256 - m
        gap_n = 256 - n
        l = gap_n // 2
        r = gap_n - l
        u = gap_m // 2
        d = gap_m - u
        image = cv2.copyMakeBorder(image, u, d, l, r, cv2.BORDER_CONSTANT, value=0)
        m = image.shape[0]
        n = image.shape[1]
    
    img_r = image[:,:,0]
    img_g = image[:,:,1]
    img_b = image[:,:,2]
    img_r = img_r.reshape((1, m*n)).T
    img_g = img_g.reshape((1, m*n)).T
    img_b = img_b.reshape((1, m*n)).T
    return img_r, img_g, img_b

def big_picture(files, path1, path2):
    flag = 0
    for file in files:
        if flag == 0:
            flag =1
            img_r, img_g, img_b = row(path1 + file)
        else:
            r, g, b = row(path1 + file)
            img_r = np.column_stack((img_r, r))
            img_g = np.column_stack((img_g, g))
            img_b = np.column_stack((img_b, b))
    np.save(path2+"1.npy",img_r.T)
    np.save(path2+"2.npy",img_g.T)
    np.save(path2+"3.npy",img_b.T)
    
def recon_image(files, big_R, big_G, big_B, save_path):
    new_files = []
    for i in range(len(files)):
        new_files.append(files[i][:-4])
    i=0
    for temp in new_files:
        r = big_R[i].reshape((256, 256)).T
        g = big_G[i].reshape((256, 256)).T
        b = big_B[i].reshape((256, 256)).T
        img = np.stack((r, g, b)).T
        imageio.imsave(save_path + temp + ".jpg", img)
        i=i+1
    
path1 = "D:/大三上/数据科学与工程算法/project1/Images/beach/"
files = os.listdir(path1)
path2 = "D:/大三上/数据科学与工程算法/project1/beach_save/"

big_picture(files, path1, path2)

r_save = path2 + "1.npy"
g_save = path2 + "2.npy"
b_save = path2 + "3.npy"

big_R = PCA(np.load(r_save), 0.95)
big_G = PCA(np.load(g_save), 0.95)
big_B = PCA(np.load(b_save), 0.95)
        
save_path = path2 + "result/"

recon_image(files, big_R, big_G, big_B, save_path)
    
            
    

398288.9885770508
0.4
410926.5160341447
0.53
394429.4839622923
0.51
